## Importing needed packages

In [1]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

## Defining classes

In [ ]:
# Class for external input data
class InputData():
    def __init__(self, CapCost, OpCost, StorCost, CapLim,CapExi,CapOut,ProdFactor,Dem,EtaCha,EtaDis,StorExi):
        self.CapCost = CapCost,
        self.OpCost = OpCost,
        self.StorCost = StorCost,
        self.CapLim = CapLim,
        self.CapExi = CapExi,
        self.CapOut = CapOut,
        self.ProdFactor = ProdFactor
        self.Dem = Dem,
        self.EtaCha = EtaCha,
        self.EtaDis = EtaDis,
        self.StorExi = StorExi

        

In [ ]:
# Class for model parameters
class InputData():
    def __init__(self, epsilon, delta ):
        self.epsilon = epsilon,
        self.delta = delta

In [ ]:
# CLASS WHICH CAN HAVE ATTRIBUTES SET

class Expando(object):
    '''
        A small class which can have attributes set
    '''
    pass

In [ ]:
# Defining the optimization model class

class CapacityProblem():
    def __init__(self, ParametersObj, DataObj, Model_results = 1, Guroby_results = 1):
        self.P = ParametersObj # Parameters
        self.D = DataObj # Data
        self.Model_results = Model_results
        self.Guroby_results = Guroby_results
        self.var = Expando()  # Variables
        self.con = Expando()  # Constraints
        self.res = Expando()  # Results
        self._build_model() 


    def _build_variables(self):
        # Create the variables
        self.var.CapNew = self.m.addMVar((self.P.N_Cap), lb=0)  # New Capacity for each type of generator technology
        self.var.EGen = self.m.addMVar((self.P.N_Cap, self.P.N_Hours, self.P.N_Scen), lb=0)  # Energy Production for each type of generator technology for each hour and scenario
        self.var.CapStor = self.m.addMVar((self.P.N_Stor), lb=0)  # New storage capacity for each type of storage technology
        self.var.SOC = self.m.addMVar((self.P.N_Stor, self.P.N_Hours, self.P.N_Scen), lb=0)  # State of charge for each type of storage technology for each hour and scenario  
        self.var.EChar = self.m.addMVar((self.P.N_Stor, self.P.N_Hours, self.P.N_Scen), lb=0)  # Energy charged for each type of storage technology for each hour and scenario
        self.var.EDis = self.m.addMVar((self.P.N_Stor, self.P.N_Hours, self.P.N_Scen), lb=0)  # Energy discharged for each type of storage technology for each hour and scenario


    def _build_constraints(self):
        # Power flow constraints, one per transmission line
        self.con.power_flow_0 = self.m.addConstr(self.var.theta @ self.P.first_zone == self.P.voltage_angle_0, name='Initial voltage angle')
        self.Inv_Trans_React = 1/self.D.Trans_React
        Delta_theta = self.var.theta @ self.D.Trans_Line_From_Z - self.var.theta @ self.D.Trans_Line_To_Z
        self.con.power_flow = self.m.addConstr(self.P.Sum_over_hours @ self.Inv_Trans_React.T * Delta_theta <= self.P.Sum_over_hours @ self.D.Trans_Cap.T, name='Power flow constraint')

        # Max demand constraint
        self.con.max_dem = self.m.addConstr(self.var.d <= self.D.Dem, name='Maximum demand')

        # Max production constraint
        self.con.max_p_E = self.m.addConstr(self.var.p_E <= self.D.Gen_E_OpCap * (self.P.Sum_over_hours @ self.D.Gen_E_Cap.T), name='Maximum production of existing generators')

        # Balance constraint
        prod_zone = self.var.p_E @ self.D.Gen_E_Z.T
        dem_zone = self.var.d @ self.D.Load_Z.T
        trans_zone = self.P.Sum_over_hours @ self.Inv_Trans_React.T * (self.var.theta - self.var.theta @ self.D.Trans_Z_Connected_To_Z.T)
        self.con.balance = self.m.addConstr(dem_zone - prod_zone == -trans_zone, name='Demand balance')
        
    
    def _build_objective(self):
        # Objective function
        objective = gp.quicksum(self.var.d @ self.D.Uti - self.var.p_E @ self.D.Gen_E_OpCost)
        self.m.setObjective(objective, GRB.MAXIMIZE)


    def _display_guropby_results(self):
        self.m.setParam('OutputFlag', self.Guroby_results)

    
    def _build_model(self):
        self.m = gp.Model('Market Clearing Problem')
        self._build_variables()
        self._build_constraints()
        self._build_objective()
        self._display_guropby_results()
        self.m.optimize()
        self._results()
        if self.Model_results == 1:
            self._extract_results()
            

    
    def _results(self):
        self.res.obj = self.m.objVal
        self.res.d = self.var.d.X
        self.res.p_E = self.var.p_E.X
        self.res.DA_price = self.con.balance.Pi

    def _extract_results(self):
        # Display the objective value
        print('Objective value: ', self.m.objVal)
        
        # Display the optimal values of the decision variables for the 24 first hours in a df, looping through the zones
        n_test = self.P.N
        self.res.df = pd.DataFrame(columns=['Hour'])
        self.res.df['Hour'] = np.arange(1, n_test + 1)
        Load = self.var.d[0:n_test].X @ self.D.Load_Z.T
        Existing_gen = self.var.p_E[0:n_test].X @ self.D.Gen_E_Z.T
        Price = self.con.balance.Pi[0:n_test]
        Trans = self.P.Sum_over_hours[0:n_test] @ self.Inv_Trans_React.T * (self.var.theta[0:n_test].X @ self.D.Trans_Line_From_Z - self.var.theta[0:n_test].X @ self.D.Trans_Line_To_Z)
        for zone in range(self.P.N_zone):
            self.res.df['Load Zone ' + str(zone)] = Load[:, zone]
            self.res.df['Existing generators Zone ' + str(zone)] = Existing_gen[:, zone]
            self.res.df['Price Zone ' + str(zone)] = Price[:, zone]
        for line in range(self.P.N_line):
            self.res.df['Power flow line ' + str(line + 1)] = Trans[:, line]